# Session Management

In this notebook, we'll explore how to manage sessions, maintain context across interactions, and use session forking to explore different approaches.

## Setup

First, let's set up our environment:

In [27]:
# Setup for running async code in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

print("✓ Notebook environment configured")

✓ Notebook environment configured


In [28]:
import os

# Verify API key
api_key = os.environ.get("ANTHROPIC_API_KEY")
if api_key:
    print(f"✓ API key found (length: {len(api_key)} characters)")
else:
    print("✗ API key not found. Please set ANTHROPIC_API_KEY environment variable.")

✓ API key found (length: 108 characters)


## Helper Function

Let's create a helper to print messages cleanly:

In [29]:
import json

def print_message(message):
    """Pretty print agent messages."""
    msg_type = type(message).__name__
    
    if msg_type == "SystemMessage":
        # Print session info from init messages
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"🔄 Session initialized: {session_id}")
            return session_id
    
    elif msg_type == "AssistantMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "TextBlock":
                    print(f"🤖 Assistant: {block.text}")
                elif block_type == "ToolUseBlock":
                    print(f"🔧 Tool: {block.name}")
                    if hasattr(block, 'input') and 'description' in block.input:
                        print(f"   → {block.input['description']}")
    
    elif msg_type == "UserMessage":
        if hasattr(message, 'content'):
            for block in message.content:
                block_type = type(block).__name__
                if block_type == "ToolResultBlock":
                    if block.is_error:
                        print(f"❌ Tool Error: {block.content}")
                    else:
                        content = str(block.content)
                        if len(content) > 300:
                            content = content[:300] + "..."
                        print(f"📤 Tool Result: {content}")
    
    elif msg_type == "ResultMessage":
        if hasattr(message, 'total_cost_usd') and hasattr(message, 'duration_ms'):
            print(f"\n💰 Cost: ${message.total_cost_usd:.4f} | ⏱️ Time: {message.duration_ms/1000:.1f}s")
    
    return None

## How Sessions Work

When you start a new query, the SDK automatically creates a session and returns a session ID in the initial system message. This session ID can be used to:

- **Resume** the conversation to continue building linearly
- **Fork** the conversation to create independent branches for exploring alternatives

### Key Concepts

- **Session ID**: Unique identifier for a conversation thread
- **Resume**: Continue the same session linearly (keeps same session ID)
- **Fork**: Create a new session that branches from the current state (generates new session ID)
- **Context**: All previous messages and tool results are maintained in each session

### The Workflow We'll Demonstrate

1. **Create** initial content (formal announcement)
2. **Fork** to try a different approach (casual version) - preserves original
3. **Resume original** session to add more content (FAQ in formal tone)
4. **Resume forked** session to add the same content (FAQ in casual tone)

This shows how fork enables you to maintain multiple independent versions that evolve separately.

## Example 1: Starting a New Session

Let's start by asking the agent to help us write a formal announcement:

In [30]:
from claude_agent_sdk import query, ClaudeAgentOptions

print("=" * 60)
print("Starting a new session")
print("=" * 60)

async def capture_session_id():
    session_id = None
    
    async for message in query(
        prompt="Write a brief announcement about our office returning to 3 days in-office policy starting next month",
        options=ClaudeAgentOptions(model="claude-sonnet-4-5")
    ):
        # Capture session ID from init message
        if hasattr(message, 'subtype') and message.subtype == 'init':
            session_id = message.data.get('session_id')
            print(f"📝 Session started with ID: {session_id}\n")
        
        # Print other messages
        print_message(message)
    
    return session_id

# Capture the session ID for later use
saved_session_id = await capture_session_id()
print(f"\n✅ Saved session ID: {saved_session_id}")

Starting a new session
📝 Session started with ID: b362698d-dcab-4eeb-aaf8-76d552352050

🔄 Session initialized: b362698d-dcab-4eeb-aaf8-76d552352050
🤖 Assistant: I can help you write a brief announcement about the return-to-office policy. Here's a professional and clear announcement:

---

**Subject: Updated In-Office Work Policy – Effective [Month, Year]**

Dear Team,

We're writing to inform you of an update to our workplace policy. Starting next month, we'll be implementing a hybrid work schedule requiring three days of in-office presence per week.

**Key Details:**
- **Effective Date:** [First day of next month]
- **Requirement:** Three days in-office, two days remote
- **Office Days:** [To be determined by team/manager or specify if company-wide]

This change is designed to foster greater collaboration, strengthen team connections, and support our continued growth while maintaining the flexibility you value.

We understand this is an adjustment, and we're committed to making this t

### What Happened?

1. The query started and created a new session
2. The first message was a `SystemMessage` with `subtype='init'`
3. We extracted the `session_id` from `message.data`
4. The agent wrote a formal email announcement
5. We saved the session ID - this will be our "main branch"

**Next**: We'll fork this session to create a casual version while keeping the formal one intact.

## Example 2: Forking to Try a Different Approach

Instead of continuing linearly, let's **fork** to create a casual version while preserving the formal one:

In [31]:
print("\n" + "=" * 60)
print("Forking to create a casual version")
print("=" * 60)

async def fork_session():
    forked_session_id = None
    
    async for message in query(
        prompt="Rewrite this announcement as a casual Slack message instead",
        options=ClaudeAgentOptions(
            resume=saved_session_id,
            fork_session=True,  # Fork to create a new branch
            model="claude-sonnet-4-5"
        )
    ):
        # Capture the new forked session ID
        if hasattr(message, 'subtype') and message.subtype == 'init':
            forked_session_id = message.data.get('session_id')
            print(f"🔀 Forked session created: {forked_session_id}")
            print(f"   Original session: {saved_session_id}")
            print(f"   Same? {forked_session_id == saved_session_id}\n")
        
        print_message(message)
    
    return forked_session_id

casual_session_id = await fork_session()
print(f"\n✅ Casual (forked) session ID: {casual_session_id}")


Forking to create a casual version
🔀 Forked session created: 684db7c5-62a2-4fa1-8ee1-462d4303364c
   Original session: b362698d-dcab-4eeb-aaf8-76d552352050
   Same? False

🔄 Session initialized: 684db7c5-62a2-4fa1-8ee1-462d4303364c
🤖 Assistant: Here's a casual Slack version:

---

Hey team! 👋

Quick heads up – starting next month, we're shifting to a 3-days-in-office schedule. This means we'll all be together in person three days a week, with two days remote.

We think this'll help us collaborate better and hang out more as a team, while still keeping some of that WFH flexibility we all appreciate.

Your manager will follow up soon with more details about which days work best for your team. In the meantime, if you have any questions or concerns, feel free to drop them in the thread or reach out directly.

Thanks everyone! 🙌

---

This version is more conversational and appropriate for Slack's informal communication style.

💰 Cost: $0.0142 | ⏱️ Time: 5.6s

✅ Casual (forked) session ID:

### What Just Happened?

- Created a **new session ID** (fork)
- The fork started with the formal announcement context
- Rewrote it in a casual tone
- **Original session is preserved** - we can still continue the formal version

Now we have TWO independent sessions:
- **Session A** (original): Formal announcement
- **Session B** (fork): Casual Slack version

## Example 3: Resume Original Session (Formal Version)

Let's continue building the **formal version** by adding an FAQ section:

In [32]:
print("\n" + "=" * 60)
print("Resuming ORIGINAL session (formal version)")
print("=" * 60)

async def resume_formal():
    async for message in query(
        prompt="Add a FAQ section with questions about parking, which days are required, and flexibility for special circumstances",
        options=ClaudeAgentOptions(
            resume=saved_session_id,  # Resume the original formal session
            model="claude-sonnet-4-5"
        )
    ):
        print_message(message)

await resume_formal()


Resuming ORIGINAL session (formal version)
🔄 Session initialized: b362698d-dcab-4eeb-aaf8-76d552352050
🤖 Assistant: Here's the updated announcement with an FAQ section:

---

**Subject: Updated In-Office Work Policy – Effective [Month, Year]**

Dear Team,

We're writing to inform you of an update to our workplace policy. Starting next month, we'll be implementing a hybrid work schedule requiring three days of in-office presence per week.

**Key Details:**
- **Effective Date:** [First day of next month]
- **Requirement:** Three days in-office, two days remote
- **Office Days:** [To be determined by team/manager or specify if company-wide]

This change is designed to foster greater collaboration, strengthen team connections, and support our continued growth while maintaining the flexibility you value.

We understand this is an adjustment, and we're committed to making this transition as smooth as possible. Your manager will reach out shortly to discuss scheduling and answer any question

### Notice:

- Used the **original session ID** (saved_session_id)
- Agent maintained the **formal tone** from the original announcement
- FAQ is professional and structured
- This session has NO knowledge of the casual Slack version

## Example 4: Resume Forked Session (Casual Version)

Now let's continue the **casual Slack version** by adding the same FAQ content but in a casual tone:

In [33]:
print("\n" + "=" * 60)
print("Resuming FORKED session (casual version)")
print("=" * 60)

async def resume_casual():
    async for message in query(
        prompt="Add a FAQ section with questions about parking, which days are required, and flexibility for special circumstances",
        options=ClaudeAgentOptions(
            resume=casual_session_id,  # Resume the forked casual session
            model="claude-sonnet-4-5"
        )
    ):
        print_message(message)

await resume_casual()


Resuming FORKED session (casual version)
🔄 Session initialized: 684db7c5-62a2-4fa1-8ee1-462d4303364c
🤖 Assistant: Here's the updated Slack message with an FAQ section:

---

Hey team! 👋

Quick heads up – starting next month, we're shifting to a 3-days-in-office schedule. This means we'll all be together in person three days a week, with two days remote.

We think this'll help us collaborate better and hang out more as a team, while still keeping some of that WFH flexibility we all appreciate.

Your manager will follow up soon with more details about which days work best for your team. In the meantime, if you have any questions or concerns, feel free to drop them in the thread or reach out directly.

**FAQ:**

**Q: Which days do I need to be in the office?**
A: This will vary by team! Your manager will work with you to figure out the best schedule. Some teams might have set days (like Tue/Wed/Thu), while others might have more flexibility depending on projects and meetings.

**Q: What 

### Notice:

- Used the **forked session ID** (casual_session_id)
- Agent maintained the **casual, friendly tone** from the Slack message
- Same FAQ content but adapted to the casual context
- This session has NO knowledge of the formal version's FAQ
